In [0]:
import tensorflow as tf
tf.__version__
!pip uninstall tensorflow -y


Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 22kB/s 
     |████████████████████████████████| 450kB 49.7MB/s 
     |████████████████████████████████| 3.8MB 56.9MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
# path to test image
test_img_path = "ILSVRC2012_val_00001110.JPEG"
# test_img_path = "test_image_1.jpg"
QUANTIZE_MODEL = True

if (QUANTIZE_MODEL):
  test_model_path = "/content/tmp_mobile_quant/test.pb"
else 
  test_model_path = "/content/mobilenet_v2_1.0_224.pb"


In [0]:
!ls

dataset  drive	imagenet2012_validation_100_labels.txt	sample_data


In [0]:
class MobileNetV2(object):

    def __init__(self, model_filepath):

        self.model_filepath = model_filepath

        # Load and Initialize model
        self.load_graph(model_filepath = self.model_filepath)


    def load_graph(self, model_filepath):

        print('Loading model...')
        self.graph = tf.Graph()

        with tf.io.gfile.GFile(test_model_path, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())
        
        with tf.Graph().as_default() as self.graph:
          tf.import_graph_def(graph_def,
                          input_map=None,
                          return_elements=None,
                          name="")
        
        # print(self.graph.get_tensor_by_name("input:0"))
        # print(self.graph.get_tensor_by_name("MobilenetV2/Logits/Conv2d_1c_1x1/BiasAdd:0"))

        #Print all operations!
        # for op in self.graph.get_operations():
        #       # print(op.name)
        #       print(self.graph.get_tensor_by_name(op.name+":0"))
        
        print('Model loading complete!')

    def predict(self, img):
        if(QUANTIZE_MODEL):
          y_pred = self.graph.get_tensor_by_name("output_fx:0")
        else:
          y_pred = self.graph.get_tensor_by_name("MobilenetV2/Logits/Conv2d_1c_1x1/BiasAdd:0")
        # y_pred = tf.reshape(y_pred, [1,1,1,1001])
        # print(y_pred.shape)
        if(QUANTIZE_MODEL):
          x= self.graph.get_tensor_by_name("input_fx:0")
        else:
          x= self.graph.get_tensor_by_name("input:0")
        sess= tf.compat.v1.Session(graph=self.graph)
        # creating the dict to calculate y_pred 
        test_dict = {x: img}
        result=sess.run(y_pred, feed_dict=test_dict)
        # print(result.shape)
        return result


In [0]:
dataset_path='/content/dataset/'
from os import listdir
from os.path import isfile, join

def listOfTuples(l1, l2): 
    return list(map(lambda x, y:(x,y), l1, l2)) 

#Get 100 images
imagenet_100=sorted(listdir(dataset_path))
print(imagenet_100)
#Get 100 labels
with open('imagenet2012_validation_100_labels.txt') as f:
    labels = f.read().splitlines()

dataset = listOfTuples(imagenet_100, labels)
print(dataset[0][0])

['ILSVRC2012_val_00000001.JPEG', 'ILSVRC2012_val_00000002.JPEG', 'ILSVRC2012_val_00000003.JPEG', 'ILSVRC2012_val_00000004.JPEG', 'ILSVRC2012_val_00000005.JPEG', 'ILSVRC2012_val_00000006.JPEG', 'ILSVRC2012_val_00000007.JPEG', 'ILSVRC2012_val_00000008.JPEG', 'ILSVRC2012_val_00000009.JPEG', 'ILSVRC2012_val_00000010.JPEG', 'ILSVRC2012_val_00000011.JPEG', 'ILSVRC2012_val_00000012.JPEG', 'ILSVRC2012_val_00000013.JPEG', 'ILSVRC2012_val_00000014.JPEG', 'ILSVRC2012_val_00000015.JPEG', 'ILSVRC2012_val_00000016.JPEG', 'ILSVRC2012_val_00000017.JPEG', 'ILSVRC2012_val_00000018.JPEG', 'ILSVRC2012_val_00000019.JPEG', 'ILSVRC2012_val_00000020.JPEG', 'ILSVRC2012_val_00000021.JPEG', 'ILSVRC2012_val_00000022.JPEG', 'ILSVRC2012_val_00000023.JPEG', 'ILSVRC2012_val_00000024.JPEG', 'ILSVRC2012_val_00000025.JPEG', 'ILSVRC2012_val_00000026.JPEG', 'ILSVRC2012_val_00000027.JPEG', 'ILSVRC2012_val_00000028.JPEG', 'ILSVRC2012_val_00000029.JPEG', 'ILSVRC2012_val_00000030.JPEG', 'ILSVRC2012_val_00000031.JPEG', 'ILSVRC

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils, mobilenet
import os
import cv2
from tensorflow.compat.v1 import enable_eager_execution

from matplotlib import pyplot as plt
%matplotlib inline

print(os.path.join(os.getcwd()))
def process_image(img_path, mobnet=False):
  if(mobnet):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    pImg = mobilenet.preprocess_input(img_array)
    # print(pImg.shape)
  else:
    from PIL import Image
    import cv2
    x = cv2.imread(img_path)
    # print(x)
    x=x[:,:,::-1]
    img = Image.fromarray(x,'RGB')
    pImg=np.array(img.resize((224,224))).astype(np.float)/128-1
    pImg = np.reshape(pImg, [1,224,224,3])
  return pImg

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

if __name__ == '__main__':
  tf.compat.v1.enable_eager_execution()
  # load model
  # model = mobilenet.MobileNet()
  model = MobileNetV2(test_model_path)

  good = []
  # process
  for image in dataset:
    # print(image)
    pImg = process_image("dataset/"+image[0])

    # predict
    prediction = model.predict(pImg)
    prediction = prediction.flatten()[:-1]
    prediction = np.reshape(prediction,[1,1000])
    results = imagenet_utils.decode_predictions(prediction, top=50)
    # print(len(results[0]))
    # print(len(results))
    for r in results[0]:
        # print(len(r),r[0])
        if (r[0]==image[1]): 
          print(r[1],r[2],image[0])

          # img = cv2.imread("dataset/"+image[0])
          # img2 = img[:,:,::-1]
          # plt.imshow(img)

/content
Loading model...
Model loading complete!
bakery 2.3125 ILSVRC2012_val_00000008.JPEG
table_lamp 2.25 ILSVRC2012_val_00000026.JPEG
